
# CVA Sensitivity using AAD

This notebook demonstrates a work in progress implementation of CVA AAD sensitivities
- in the context of the scripted trade framework
- for a single Swap, as in Example 1

We highlight performance of the calculation and show differences between AAD and bump & revalue sensitivities.


## Recall: Run CVA using "classic" AMC
- AMC simulation generates the NPV cube, biweekly grid to 10 years, 8192 paths
- post processor step aggregates the expected exposure and copmputes CVA

In [ ]:
from ORE import *
import sys, time, math
sys.path.append('..')
import utilities

In [ ]:
params_xva = Parameters()
params_xva.fromFile("Input/ore_amc_legacy.xml")
ore_xva = OREApp(params_xva)
ore_xva.run()
utilities.checkErrorsAndRunTime(ore_xva)
utilities.writeList(ore_xva.getReportNames())

In [ ]:
# plot the exposure report
exposure_report = ore_xva.getReport("exposure_nettingset_CPTY_A")
time = exposure_report.dataAsReal(2)
epe = exposure_report.dataAsReal(3)
ene = exposure_report.dataAsReal(4)

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 5))
plt.plot(time, epe, label='EPE')
plt.plot(time, ene, label='ENE')
plt.xlabel('Time/Years')
plt.ylabel('Exposure')
plt.title('')
plt.show()

In [ ]:
# show the xva report
xva_report = ore_xva.getReport("xva")
display(utilities.format_report(xva_report))

## Run CVA on the Computation Graph
We use a proof of concept implementation (see orea/engine/xvaenginecg.xpp) called from the XVA analytic (see orea/app/analytics/xvaanalytic.cpp) if parameter **amcCg=true** in ore_ad.xml.

This engine performes AMC valuation, exposure aggregation and XVA calculation in one shot on ORE's ComputationGraph. 

We can force calculation of sensitivities by bump & revalue if parameter **xvaCgBumpSensis=true**, see ore_bump.xml. This is what we do below, to generate "reference" sensitivities.


In [ ]:
params = Parameters()
params.fromFile("Input/ore__cvasensi_bump.xml")
ore = OREApp(params)
ore.run()
utilities.checkErrorsAndRunTime(ore)
utilities.writeList(ore.getReportNames())

In [ ]:
exposure = ore.getReport("xvacg-exposure")
display(utilities.format_report(exposure))

In [ ]:
sensi_scenario_bump = ore.getReport("xvacg-cva-sensi-scenario")
display(utilities.format_report(sensi_scenario_bump))

## Run CVA Sensitivity using AAD

In [ ]:
params_ad = Parameters()
params_ad.fromFile("Input/ore__cvasensi_ad.xml")
ore_ad = OREApp(params_ad)
ore_ad.run()
utilities.checkErrorsAndRunTime(ore_ad)

In [ ]:
# Compare Sensitivity Scenarios
utilities.match_scenario_reports(ore, ore_ad, False)

## Summary

Swap example performance (Macbook Pro, Apple M2 Max 2023) 
- ~9 seconds run time for CVA with "classic" AMC simulation and postprocessing for exposure and CVA
- ~44 seconds run time for CVA Sensitivities, ComputationGraph, Bump & Reval
- ~5 seconds run time for CVA Sensitivities with AAD